<a href="https://colab.research.google.com/github/Chauhanshi/Predictive-Analytics/blob/master/optimize_CNN_MNIST_fashion_item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=ef23e9ac87634e3b4b84e4ed70e41fe8d0a51892505bbe4a57412fa862fc806c
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ac67b6f3ce14c9e1b49cd7c643ea00ff5aca3c9783b938f333c284bfe75b88cc
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
train_images[0].shape

(28, 28, 1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3846 - accuracy: 0.8611 - val_loss: 0.2804 - val_accuracy: 0.8958
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2402 - accuracy: 0.9107 - val_loss: 0.2431 - val_accuracy: 0.9112
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1809 - accuracy: 0.9339 - val_loss: 0.2427 - val_accuracy: 0.9107


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4357 - accuracy: 0.8427 - val_loss: 0.3498 - val_accuracy: 0.8727
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3155 - accuracy: 0.8837 - val_loss: 0.3225 - val_accuracy: 0.8862
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2757 - accuracy: 0.8994 - val_loss: 0.3500 - val_accuracy: 0.8832


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4862 - accuracy: 0.8313 - val_loss: 0.4307 - val_accuracy: 0.8540
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3454 - accuracy: 0.8710 - val_loss: 0.3560 - val_accuracy: 0.8708
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3083 - accuracy: 0.8853 - val_loss: 0.3424 - val_accuracy: 0.8792


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4826 - accuracy: 0.8358 - val_loss: 0.3726 - val_accuracy: 0.8590
Epoch 2/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3510 - accuracy: 0.8713 - val_loss: 0.3797 - val_accuracy: 0.8593
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3266 - accuracy: 0.8792 - val_loss: 0.3725 - val_accuracy: 0.8638


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5260 - accuracy: 0.8240 - val_loss: 0.3988 - val_accuracy: 0.8500
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3788 - accuracy: 0.8602 - val_loss: 0.3848 - val_accuracy: 0.8565
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3506 - accuracy: 0.8689 - val_loss: 0.3962 - val_accuracy: 0.8543


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,676,538
Trainable params: 2,676,538
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1810 - accuracy: 0.9327 - val_loss: 0.2421 - val_accuracy: 0.9112
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1310 - accuracy: 0.9503 - val_loss: 0.2533 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0912 - accuracy: 0.9665 - val_loss: 0.2754 - val_accuracy: 0.9118
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0606 - accuracy: 0.9775 - val_loss: 0.3499 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0461 - accuracy: 0.9834 - val_loss: 0.3843 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0322 - accuracy: 0.9881 - val_loss: 0.4572 - val_accuracy: 0.9080
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0253 - accuracy: 0.9912 - val_loss: 0.4339 - val_accuracy